In [ ]:
import geemap 
import ee 
ee.Initialize()
#ee.Authenticate()

In [ ]:
Map = geemap.Map()
#roi01 = geemap.shp_to_ee(r"F:\随机森林\土壤湿度反演\region\area_gcs.shp").geometry()
roi = ee.Geometry.Rectangle([113, 26, 119, 30], None, False)
Map.addLayer(roi, {}, 'Boundary')


In [ ]:
startDate = ee.Date.fromYMD(2023,1,1)
endDate = ee.Date.fromYMD(2023,12,31)
#Sentinel-2 去云
def maskCloudAndShadowsSR(image):
  cloudProb = image.select('MSK_CLDPRB')
  snowProb = image.select('MSK_SNWPRB')
  cloud = cloudProb.lt(5)
  snow = snowProb.lt(5)
  scl = image.select('SCL')
  shadow = scl.eq(3); # 3 = cloud shadow
  cirrus = scl.eq(10); # 10 = cirrus
  # Cloud probability less than 5% or cloud shadow classification
  mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
  return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])
#function: EVI for S2
def funEVIS2(image,B1,B2,B3):
  VI = image.expression('2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
      {
        'blue': image.select(B1),
        'red':  image.select(B2),
        'nir':  image.select(B3),
      }).rename('EVI')
  return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
def addVIs(img):
  nir = img.select('B8')
  red = img.select('B4')
  blue = img.select('B2')
  evi = funEVIS2(img,'B2','B4','B8')
  ndvi = img.normalizedDifference(['B8','B4']).rename('NDVI')
  lswi = img.normalizedDifference(['B8','B11']).rename('LSWI')
  mndwi = img.normalizedDifference(['B3','B11']).rename('mNDWI')
  snow = mndwi.gt(0.4).And(nir.gt(0.11))
  img =img.addBands(evi).addBands(ndvi).addBands(lswi).addBands(mndwi).updateMask(snow.eq(0))
  # return img.select(['LSWI','EVI','NDVI','mNDWI'])
  return img.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','LSWI','EVI','NDVI','mNDWI'])
#纹理信息提取
def texture(image):
  text = image.divide(10000).toInt32().glcmTexture() #归一化
  texture_feature = text.select(['NDVI_contrast','NDVI_corr','NDVI_idm','NDVI_ent'])
  return image.addBands(texture_feature)
#修改
def Interval_Mean(image,down,up,str_down,str_up):
  indicate_bands_IM = image.reduce(ee.Reducer.intervalMean(ee.Number(down),ee.Number(up)));
  common_band = indicate_bands_IM.bandNames().map(function(x) {return ee.String(x).cat(str_down).cat(str_up);});
  return indicate_bands_IM.select(indicate_bands_IM.bandNames(),common_band)


def indicate_function(image):
  max = image.reduce(ee.Reducer.max())
  min = image.reduce(ee.Reducer.min())
  stdDev = image.reduce(ee.Reducer.stdDev())
  median = image.reduce(ee.Reducer.median())
  
  return max.addBands(min).addBands(stdDev).addBands(median)
          .addBands(Interval_Mean(image,0,10,'_0','_10'))
          .addBands(Interval_Mean(image,10,25,'_10','_25'))
    .addBands(Interval_Mean(image,25,50,'_25','_50'))
    .addBands(Interval_Mean(image,50,75,'_50','_75'))
    .addBands(Interval_Mean(image,75,90,'_75','_90'))
    .addBands(Interval_Mean(image,90,100,'_90','_100'))
    .addBands(Interval_Mean(image,10,90,'_10','_90'))
    .addBands(Interval_Mean(image,25,75,'_25','_75'))




In [ ]:

//////  modify
function Interval_Mean(image,down,up,str_down,str_up)
{
  var indicate_bands_IM = image.reduce(ee.Reducer.intervalMean(ee.Number(down),ee.Number(up)));
  var common_band = indicate_bands_IM.bandNames().map(function(x) {return ee.String(x).cat(str_down).cat(str_up);});
  return indicate_bands_IM.select(indicate_bands_IM.bandNames(),common_band);
}

function indicate_function(image){
  var max = image.reduce(ee.Reducer.max());
  var min = image.reduce(ee.Reducer.min());
  var stdDev = image.reduce(ee.Reducer.stdDev());
  var median = image.reduce(ee.Reducer.median());
  
  return max.addBands(min).addBands(stdDev).addBands(median)
            .addBands(Interval_Mean(image,0,10,'_0','_10'))
            .addBands(Interval_Mean(image,10,25,'_10','_25'))
            .addBands(Interval_Mean(image,25,50,'_25','_50'))
            .addBands(Interval_Mean(image,50,75,'_50','_75'))
            .addBands(Interval_Mean(image,75,90,'_75','_90'))
            .addBands(Interval_Mean(image,90,100,'_90','_100'))
            .addBands(Interval_Mean(image,10,90,'_10','_90'))
            .addBands(Interval_Mean(image,25,75,'_25','_75'));
}

function texture_spectral_function(image)
{
  return Interval_Mean(image,10,90,'_10','_90');
}

/////////////////////////////////////Train the classifier using data 2022///////////////////////////////////////////////
var empty = ee.Image().byte();
var outline = empty.paint({
  featureCollection: southLine,
  color: 'yellow',
  width: 3
});
Map.addLayer(outline,{},"outline",false);

//////Data preparation//////
var s2Sr = ee.ImageCollection('COPERNICUS/S2_SR');

// filter data and location
var images = s2Sr.filterDate(startDate, endDate)
                     .filterBounds(southLine)
                     .map(maskCloudAndShadowsSR)
                     .map(addVIs)
                     .map(texture);
  
print('Sentinel-2 Level 2');
// print(images.size());
// print(images.first().bandNames())

// ////Step 1: Bands management////////
// var composite = images.reduce(ee.Reducer.percentile([10,50,90]));

// // print("composite",composite.bandNames());
// //Input variables
// var texture_bands = composite.bandNames().slice(39,51);// 24 or 51
// var texture_image = composite.select(texture_bands);
// var spectral_bands = composite.bandNames().slice(0,39);//12 or 39
// var spectral_image = composite.select(spectral_bands);
// var feature_bands = composite.bandNames();
// var feature_image = spectral_image.addBands(texture_image);
// // print(spectral_bands)
// // print(texture_bands)
// // print(feature_bands)

//////  modify
var texture_bands = images.first().bandNames().slice(-4);
var texture_image = texture_spectral_function(images.select(texture_bands));
var spectral_bands = images.first().bandNames().slice(0,9);
var spectral_image = texture_spectral_function(images.select(spectral_bands));
var indicate_bands = images.first().bandNames().slice(9,-4);
var indicate_image = indicate_function(images.select(indicate_bands));
var feature_image = spectral_image.addBands(texture_image).addBands(indicate_image);
var feature_bands = feature_image.bandNames();
// print(texture_image.bandNames());
// print(spectral_image.bandNames());
// print(indicate_image.bandNames());
print('feature_bands',feature_bands);

//////////Step2: training data////////////////
// var all_training = Mangrove_sample.merge(Saltmarsh_sample);
// // var validation = Spartina_vad.merge(Nonspartina_vad);
// print('all_training',all_training.size());
var training = Mangrove_sample.merge(Inland_sample).filterBounds(southLine);
print('training',training.size());

var training_samples = feature_image.sampleRegions({
  collection: training,
  properties: ['Class'],
  scale: 10,
  tileScale: 16
});

//print(training_samples.size(),testing_samples.size());
var trainingData = training_samples.randomColumn('random');//
var sample_training = trainingData.filter(ee.Filter.lte("random", 0.7));
var sample_validate = trainingData.filter(ee.Filter.gt("random", 0.7));
// print(sample_training.size())
// print(sample_validate.size())
// // Export the image sample feature collection to Drive as a shapefile.
// Export.table.toDrive({
//   collection: sample_training,
//   description: 'Vietnam_tidal_vegetation_training_sample',
//   folder: 'Wetlands',
//   fileFormat: 'SHP'
// });
// // Export the image sample feature collection to Drive as a shapefile.
// Export.table.toDrive({
//   collection: sample_validate,
//   description: 'Vietnam_tidal_vegetation_validate_sample',
//   folder: 'Wetlands',
//   fileFormat: 'SHP'
// });

var classifier = ee.Classifier.smileRandomForest(50).train({ 
  features: trainingData, 
  classProperty: 'Class', 
  inputProperties: feature_bands
  });
  
// //对sentinel进行分类
var class_img = feature_image.clip(southLine).classify(classifier);
// Map.addLayer(class_img.clip(southLine).updateMask(Guangxi_veget), {min:1,max:2, palette: ['yellow', 'green']},'original_veget',false)


// //Step 4: Assess
// var explain = classifier.explain();
// print('Explain',explain);

//Save the classifier
var trees = ee.List(ee.Dictionary(classifier.explain()).get('trees'));
var dummy = ee.Feature(southLine.geometry(),null);
var colTree = ee.FeatureCollection(trees.map(function(x){return dummy.set('tree',x)}));
//print(colTree);
Export.table.toAsset(colTree,'PapuaNewGuinea_tidal_vegetation_save_classifier2022','projects/geemap-2022/assets/PV_classifier');
